# N2O forcing


From O'Connor et al. 2021:

- +0.25 W m-2

no account how this breaks down into direct forcing and ozone contributions in any of Fiona's papers, so estimate the ozone contribution in 05 and the residual is direct forcing.

In [ ]:
from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

import matplotlib.pyplot as pl
import pandas as pd
import pooch

In [ ]:
climate_response_df = pd.read_csv('../data/fair-calibrations/4xCO2_energy_balance_ebm3.csv')

In [ ]:
f = FAIR(temperature_prescribed=True)

In [ ]:
scenarios = ['ssp245']
configs = ['UKESM1-0-LL']
species = [
    'CH4',
    'N2O',
    'CO2',
    'Equivalent effective stratospheric chlorine',
    'VOC',
    'NOx',
    'BC', 
    'OC', 
    'Sulfur', 
    'Ozone',
    'Aerosol-radiation interactions', 
    'Aerosol-cloud interactions'
]
species, properties = read_properties(species=species)
properties['CH4']['input_mode'] = 'concentration'
properties['N2O']['input_mode'] = 'concentration'
properties['CO2']['input_mode'] = 'concentration'
properties['Equivalent effective stratospheric chlorine']['input_mode'] = 'concentration'
properties

In [ ]:
f.define_time(1850, 2015, 1)
f.define_scenarios(scenarios)
f.define_configs(configs)
f.define_species(species, properties)

In [ ]:
f.allocate()

In [ ]:
fill(f.climate_configs['ocean_heat_capacity'], climate_response_df.loc[0, 'C1':'C3'])
fill(f.climate_configs['ocean_heat_transfer'], climate_response_df.loc[0, 'kappa1':'kappa3'])
fill(f.climate_configs['deep_ocean_efficacy'], climate_response_df.loc[0, 'epsilon'])
fill(f.climate_configs['gamma_autocorrelation'], climate_response_df.loc[0, 'gamma'])

In [ ]:
initialise(f.forcing, 0)
initialise(f.temperature, 0)

In [ ]:
f.fill_species_configs()

In [ ]:
rcmip_emissions_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-emissions-annual-means-v5-1-0.csv",
    known_hash="md5:4044106f55ca65b094670e7577eaf9b3",
)
emis_df = pd.read_csv(rcmip_emissions_file)

In [ ]:
rcmip_concentration_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-concentrations-annual-means-v5-1-0.csv",
    known_hash="md5:0d82c3c3cdd4dd632b2bb9449a5c315f",
)
conc_df = pd.read_csv(rcmip_concentration_file)

In [ ]:
bc = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|BC')&(emis_df['Region']=='World'),'1850'].values[0]
oc = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|OC')&(emis_df['Region']=='World'),'1850'].values[0]
so2 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|Sulfur')&(emis_df['Region']=='World'),'1850'].values[0]
voc = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|VOC')&(emis_df['Region']=='World'),'1850'].values[0]
nox = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|NOx')&(emis_df['Region']=='World'),'1850'].values[0]

co2 = conc_df.loc[(conc_df['Scenario']=='historical')&(conc_df['Variable']=='Atmospheric Concentrations|CO2')&(conc_df['Region']=='World'),'1850'].values[0]
n2o = conc_df.loc[(conc_df['Scenario']=='ssp245')&(conc_df['Variable']=='Atmospheric Concentrations|N2O')&(conc_df['Region']=='World'),'1850':'2015'].values.squeeze()
n2o[-2] = n2o[-1]
ch4 = conc_df.loc[(conc_df['Scenario']=='historical')&(conc_df['Variable']=='Atmospheric Concentrations|CH4')&(conc_df['Region']=='World'),'1850'].values[0]

In [ ]:
bc, oc, so2, voc, nox

In [ ]:
# 1850 baselines
f.species_configs['baseline_emissions'].loc[dict(specie='Sulfur')] = so2
f.species_configs['baseline_emissions'].loc[dict(specie='BC')] = bc
f.species_configs['baseline_emissions'].loc[dict(specie='OC')] = oc
f.species_configs['baseline_emissions'].loc[dict(specie='VOC')] = voc
f.species_configs['baseline_emissions'].loc[dict(specie='NOx')] = nox

f.species_configs['baseline_concentration'].loc[dict(specie='CH4')] = ch4
f.species_configs['baseline_concentration'].loc[dict(specie='N2O')] = n2o[0]
f.species_configs['baseline_concentration'].loc[dict(specie='CO2')] = co2
f.species_configs['baseline_concentration'].loc[dict(specie='Equivalent effective stratospheric chlorine')] = 0

In [ ]:
# fill emissions and concentrations
f.emissions.loc[dict(specie='Sulfur')] = so2
f.emissions.loc[dict(specie='BC')] = bc
f.emissions.loc[dict(specie='OC')] = oc
f.emissions.loc[dict(specie='VOC')] = voc
f.emissions.loc[dict(specie='NOx')] = nox

f.concentration.loc[dict(specie='CH4')] = ch4
f.concentration.loc[dict(specie='N2O')] = n2o[:, None, None]
f.concentration.loc[dict(specie='CO2')] = co2
f.concentration.loc[dict(specie='Equivalent effective stratospheric chlorine')] = 0

f.temperature[:] = 0

# put ARI calibrations here. There is no evidence that N2O contributes to ARI.
f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='N2O')] = 0

# pre-calibrated for aerosol-only runs: do not adjust
f.species_configs['aci_scale'][:] = -0.735769463
f.species_configs['aci_shape'].loc[dict(specie="Sulfur")] = 0.037031928
f.species_configs['aci_shape'].loc[dict(specie="BC")] = 2.84E-31
f.species_configs['aci_shape'].loc[dict(specie="OC")] = 0.016231426

# pre-calibrated for ozone runs: do not adjust
f.species_configs['ozone_radiative_efficiency'].loc[dict(specie="N2O")] = 0.0008

In [ ]:
# here comes the trial and error for component-wise tuning
f.species_configs['forcing_scale'].loc[dict(specie="N2O")] = 1.0632
# f.species_configs['ozone_radiative_efficiency'].loc[dict(specie="CH4")] = 0.00012705
# f.species_configs['h2o_stratospheric_factor'].loc[dict(specie="CH4")] = 7/54
# f.species_configs['erfari_radiative_efficiency'].loc[dict(specie="CH4")] = -5.093e-5
# f.species_configs['aci_shape'].loc[dict(specie="CH4")] = -0.00036232

In [ ]:
f.run()

In [ ]:
pl.plot(f.timebounds, f.forcing_sum[:,0,0])

In [ ]:
print('N2O               %+7.5f' % f.forcing.values[-1,0,0,1])
print('Ozone             %+7.5f' % f.forcing.values[-1,0,0,9])
print('ARI               %+7.5f' % f.forcing.values[-1,0,0,10])
print('ACI               %+7.5f' % f.forcing.values[-1,0,0,11])
print('Sum of components %+7.5f' % (f.forcing.values[-1,0,0,1]+f.forcing.values[-1,0,0,9]+f.forcing.values[-1,0,0,10]+f.forcing.values[-1,0,0,11]))
print('Scenario sum      %+7.5f' % f.forcing_sum.values[-1,0,0])

In [ ]:
f.forcing.values[-1,0,0,:]